TODO: build pipeline with mixed workflow.<br>
Example: collect data, define basic condition for next step, process next step, then next step, etc.

In [8]:
def running_avg():
    print('Start running average')
    number = 0
    numbers = 0
    total = 0
    while True:
        total += yield number
        numbers += 1
        avg = total/numbers
        print('The current average is:', avg)

In [18]:
avger = running_avg()
avger.send(None)
avger.send(10)
avger.send(6)
avger.send(29)
avger.close()
avger.send(29)


Start running average
The current average is: 10.0
The current average is: 8.0
The current average is: 15.0


StopIteration: 

In [22]:
def finite_coroutine():
    print('Coroutine starts')
    try:
        value = yield
    except GeneratorExit:
        print('Being closed')
        return
    print('Receive value {} and stops'.format(value))
    
coro = finite_coroutine()
coro.send(None)
coro.close()

Coroutine starts
Being closed


In [18]:

def proxy():
    print('proxy is started')
    result = yield from another_proxy()
    print('Resume to proxy with result', result)
    yield result
    print ('Resume to proxy again')
    
    
def another_proxy():
    print('another_proxy is started')
    result = yield from real_gen()
    print('Resume to another_proxy with result', result)
    print("I don't want to return yet, waiting here for value")
    own_value = yield
    print("Resume again to another_proxy with result:", own_value)
    return result

def real_gen():
    print("I'm the real deal")
    while True:
        result = yield
        if result is None:
            return 'inner'
        
        print('Getting value from client:', result)

prx = proxy()
next(prx)
result = prx.send('value')
print(result)
result = prx.send('Another value')
prx.throw(ValueError)
result = prx.send(None)
result = prx.send('value again')


proxy is started
another_proxy is started
I'm the real deal
Getting value from client: value
None
Getting value from client: Another value


ValueError: 

In [19]:
def normal_gen():
    print("start gen")
    yield
    print("result gen")
    print("do something")
    
gen = normal_gen()
next(gen)
gen.throw(ValueError)
next(gen)

start gen


ValueError: 

In [20]:
from concurrent import futures
from time import sleep

def lazy_task(msg):
    print(futures.thread.threading.current_thread())
    print('Going to sleep first')
    sleep(2)
    print('Printing message:', msg)
    sleep(2)
    print('Exiting,', msg)
    
    return msg + ' is completed'

def show(fs):
    print(futures.thread.threading.current_thread())
    print(fs)
    for future in fs:
        print('Trying to print result from future:', future)
        print(future.result())
        
def show_as_completed(fs):
    print(futures.thread.threading.current_thread())
    print(fs)
    for future in futures.as_completed(fs):
        print('>>>>>>>>>>>Trying to print result from future:', future)
        print(future.result())
        
msg = ['Hello', 'There', 'I', 'Love', 'You']
fs = []
print(futures.thread.threading.current_thread())
with futures.ThreadPoolExecutor(max_workers=6) as executor:
    for m in msg:
        future = executor.submit(lazy_task, m)
        print(m, '- Future submitted:', future)
        fs.append(future)
        
    executor.submit(show_as_completed, fs)


<_MainThread(MainThread, started 21664)>
<Thread(ThreadPoolExecutor-15_0, started daemon 22632)>
Going to sleep first
Hello - Future submitted: <Future at 0x1ce3edabf70 state=running>
<Thread(ThreadPoolExecutor-15_1, started daemon 7112)>There - Future submitted: <Future at 0x1ce3e7e5f70 state=running>
<Thread(ThreadPoolExecutor-15_2, started daemon 22744)>
Going to sleep first
I
Going to sleep first
 - Future submitted: <Future at 0x1ce3e8a5a60 state=running>
<Thread(ThreadPoolExecutor-15_3, started daemon 8096)>Love - Future submitted: <Future at 0x1ce3ef398b0 state=running>

Going to sleep first
<Thread(ThreadPoolExecutor-15_4, started daemon 10620)>
Going to sleep first
You - Future submitted: <Future at 0x1ce3ea72e50 state=running>
<Thread(ThreadPoolExecutor-15_5, started daemon 19288)>
[<Future at 0x1ce3edabf70 state=running>, <Future at 0x1ce3e7e5f70 state=running>, <Future at 0x1ce3e8a5a60 state=running>, <Future at 0x1ce3ef398b0 state=running>, <Future at 0x1ce3ea72e50 state=r

In [11]:
futures.thread.threading.current_thread().

<_MainThread(MainThread, started 21664)>

We will write a data transformation chain from generator and coroutine.

The goal is to easily attach/detach a particular logic.

Let's start simple with two rules: for each string in the sequence we will convert to lower case and capitalize the first letter.

Example: given the sequence ('aNaa', 'TONY', 'barBarrA') we will have ('Anaa', 'Tony', 'Barbarra')

In [25]:

                
def to_lower_case(next_chain=None):
    print('to_lower_case started')
    result = None
    if next_chain:
        next(next_chain)
    while True:
        try:
            text = yield result
        except GeneratorExit:
            if next_chain:
                next_chain.close()
            print('to_lower_case exit')
            return
        else:
            result = str.lower(text)
            if next_chain:
                result = next_chain.send(result)
        
def snake_case(next_chain=None):
    print('snake_case started')
    result = None
    if next_chain:
        next(next_chain)
    while True:
        try:
            text = yield result
        except GeneratorExit:
            if next_chain:
                next_chain.close()
            print('snake_case exits')
            return
        else:
            result = str.upper(text[0]) + text[1:]
            if next_chain:
                result = next_chain.send(result)
        
        
def main():
    seq = ['aNaa', 'TONY', 'barBarrA']
    transformers = [to_lower_case, snake_case]
    results = []
    
    transformer = None
    for i in reversed(transformers):
        transformer = i(next_chain=transformer)
        
    next(transformer)
    for text in seq:
        results.append(transformer.send(text))
    
    print(results)
    transformer.close()
        
main()

to_lower_case started
snake_case started
['Anaa', 'Tony', 'Barbarra']
snake_case exits
to_lower_case exit


In [26]:
def chain(transform_function):
    def transformer(next_chain=None):
        print('to_lower_case started')
        result = None
        if next_chain:
            next(next_chain)
        while True:
            try:
                text = yield result
            except GeneratorExit:
                if next_chain:
                    next_chain.close()
                print('to_lower_case exit')
                return
            else:
                result = transform_function(text)
                if next_chain:
                    result = next_chain.send(result)
                    
    return transformer

def main():
    seq = ['aNaa', 'TONY', 'barBarrA']
    transformers = [chain(str.lower), chain(lambda text: str.upper(text[0]) + text[1:])]
    results = []
    
    # Build the pipeline
    transformer = None
    for i in reversed(transformers):
        transformer = i(next_chain=transformer)
        
    # Prime the pipeline
    next(transformer)
    
    # Execute pipeline
    for text in seq:
        results.append(transformer.send(text))
    
    print(results)
    transformer.close()
        
main()

to_lower_case started
to_lower_case started
['Anaa', 'Tony', 'Barbarra']
to_lower_case exit
to_lower_case exit


with the `chain` function, we could reuse all the boilerplate code in each coroutine. Each coroutine now is different by only the `transform_function`.

It's super easy to write new business rule to further transform this stream of data. It's also easy to create different combination of transformer.

Let's take a moment to think if we overengineered this pipeline? Could we achieve the same with simpler feature such as map? or perhap a while loop?
Yes we can.

In [29]:
seq = ['aNaa', 'TONY', 'barBarrA']
results = []
for text in seq:
    text = text.lower()
    text = text[0].upper() + text[1:]
    results.append(text)

print(results)

['Anaa', 'Tony', 'Barbarra']


In [40]:
seq = ['aNaa', 'TONY', 'barBarrA']
results = map(str.lower, seq)
results = map(lambda text: str.upper(text[0]) + text[1:], results)

print(list(results))

['Anaa', 'Tony', 'Barbarra']


In [46]:
# Pipeline with map
seq = (i for i in ['aNaa', 'TONY', 'barBarrA'])
transformers = [str.lower, lambda text: str.upper(text[0]) + text[1:]]

results = seq
for transformer in transformers:
    results = map(transformer, results)
    
list(results)

['Anaa', 'Tony', 'Barbarra']

The thing with the `for loop` is that it's not convenient to add/remove logic. While the issue with `map` is that it only works on a finite set of values. If we have a continuous stream of data, `map` will keep working without yielding any results.

In [ ]:
# Pipeline with map

import itertools


seq = (i for i in itertools.cycle(['aNaa', 'TONY', 'barBarrA']))
transformers = [str.lower, lambda text: str.upper(text[0]) + text[1:]]

results = seq
for transformer in transformers:
    results = map(transformer, results)
    
list(results)

In [ ]:
import itertools


def chain(transform_function):
    def transformer(next_chain=None):
        print('to_lower_case started')
        result = None
        
        # Check if there's another coro downstream, if so, prime it
        if next_chain:
            next(next_chain)
            
        while True:
            try:
                text = yield result
            except GeneratorExit:
                if next_chain:
                    next_chain.close()
                print('to_lower_case exit')
                return
            else:
                result = transform_function(text)
                # Pass the value to next coro
                if next_chain:
                    result = next_chain.send(result)
                    
    return transformer

def main():
    seq = (i for i in itertools.cycle(['aNaa', 'TONY', 'barBarrA']))
    transformers = [chain(str.lower), chain(lambda text: str.upper(text[0]) + text[1:])]
    results = []
    
    # Build the pipeline
    transformer = None
    for i in reversed(transformers):
        transformer = i(next_chain=transformer)
        
    # Prime the pipeline
    next(transformer)
    
    # Execute pipeline
    for text in seq:
        result = transformer.send(text)
        print(result)
    
    print(results)
    transformer.close()
        
main()

With the coroutine pipeline, the processed string is yielded one by one. This is definitely usefull when we have to deal with an infinite stream or a large amount of data, we don't have to load everything at once.

It's also highly configurable so you could separate business logic from the flow, making your code easy to maintain and expand.

This implementation is an example of `push` style where we push each value through the pipeline.

Next, we will try the same with `generator` in the pulling style.

In [8]:
import itertools


def gen(transform_function, seq):
    for text in seq:
        result = transform_function(text)
        yield result
        
def snake_case(text):
    return text[0].upper() + text[1:]
        
def main():
    seq = (i for i in itertools.cycle(['aNaa', 'TONY', 'barBarrA']))
    transformer = [str.lower, snake_case]
    my_gen = seq
    for transformer in transformer:
        my_gen = gen(transformer, my_gen)
        
    for i in my_gen:
        print(i)

main()

Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Ba

Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Ana

Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarr

Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarr

Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Ba

Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Ana

Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarr

Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Barbarra
Anaa
Tony
Ba

KeyboardInterrupt: 

Looks much more simpler then the coroutine implementation yet still achieve the same result:
1. It's easy to config the pipeline.
2. It works with unfinite stream and large data.
3. It doesn't require too much boiler plate code like coroutine.

>***NOTE:***<br>
It's worthwhile to highlight another difference between `pull` and `push` style: the order in which transformation functions are built into pipeline.<br>
If we have a pipeline of 3 transformation:<br>
input > transformation 1 > transformation 2 > transformation 3 > output<br>
in `push` style, we, well, push the input through 1 > 2 > 3<br>
in `pull` style, we demand a value from 3 which demands a value from 2 which demans a value from 1.<br>
In the scenario where the order of these transformations is critical (as in our example, we need to lower case first before capitalizing the first letter), you may need to be mindful about the actual order these transformation will be executed in each style, i.e. `pulling` or `pushing`.

I believe we could conclude that for a simple data transformation pipeline, `generator` rocks. In this aspect, the real power of `coroutine` doesn't shine since its power lies mostly in supporting concurrency.